In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import mnist
from keras.layers import BatchNormalization
from sklearn.svm import SVC
import theano
from keras.utils import np_utils
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.engine import base_layer_utils
from tensorflow.python.keras.engine import input_spec
from tensorflow.python.keras.engine import keras_tensor
from tensorflow.python.keras.engine import node as node_module
from keras import backend 
import keras
import tensorflow as tf

In [19]:
def svc(traindata,trainlabel,testdata,testlabel,fin_test):
    print("Start training SVM...")
    svcClf = SVC(C=8,kernel="rbf",cache_size=3000)
    svcClf.fit(traindata,trainlabel)

    pred_testlabel = svcClf.predict(testdata)
    fin_testlabel=svcClf.predict(fin_test)
    num = len(pred_testlabel)
    print(type(pred_testlabel))
    print(type(testlabel))
    accuracy = len([1 for i in range(num) if testlabel[i]==pred_testlabel[i]])/float(num)
    print("cnn-svm Accuracy:",accuracy)
    print(pred_testlabel)
    df_svcClf_out=pd.DataFrame({
    "RowNumber":df_test['RowNumber'],
    "Exited":fin_testlabel
    })
    df_svcClf_out.to_csv('keras_svcClf.csv')

In [13]:
df_test=pd.read_csv('test.csv')
dx2=df_test[['CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']]
dx_test=pd.get_dummies(dx2,columns=['NumOfProducts'])
dx_test -= dx_test.mean(axis=0)
dx_test=dx_test.values
print(dx_test)
dx_test -= dx_test.mean(axis=0)
dx_test /= dx_test.std(axis=0)

[[ 9.67545e+01 -3.85250e+00 -1.52350e+00 ... -4.46500e-01 -2.55000e-02
  -5.50000e-03]
 [ 3.17545e+01 -8.52500e-01 -1.52350e+00 ... -4.46500e-01 -2.55000e-02
  -5.50000e-03]
 [-8.42455e+01 -8.85250e+00  1.47650e+00 ...  5.53500e-01 -2.55000e-02
  -5.50000e-03]
 ...
 [-8.12455e+01  3.14750e+00 -1.52350e+00 ...  5.53500e-01 -2.55000e-02
  -5.50000e-03]
 [-2.72455e+01  1.14750e+00  2.47650e+00 ...  5.53500e-01 -2.55000e-02
  -5.50000e-03]
 [-1.42455e+01 -1.08525e+01  4.76500e-01 ... -4.46500e-01 -2.55000e-02
  -5.50000e-03]]


In [4]:
df = pd.read_csv("train.csv")

In [5]:
dx_train=df['Exited']
labels=df.Exited.values
feature=df[['CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']]
feature_train=pd.get_dummies(feature,columns=['NumOfProducts'])
dataset=StandardScaler().fit_transform(feature_train)
#dataset=feature_train.values
labels=df.Exited.values
print(len(labels))

8000


In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset,labels,test_size=0.2)
print(Y_train)

[0 0 1 ... 0 0 0]


In [7]:
model = Sequential()
model.add(Dense(12, input_shape=(11,), activation="relu"))
model.add(Dense(12, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam",
metrics=["accuracy"])
model.fit(X_train, Y_train, validation_split=0.33,epochs=150, batch_size=10, verbose=0)
Y_pred = model.predict_classes(dx_test, batch_size=10, verbose=0)
Y_pred=Y_pred.flatten()
df_Clf_out=pd.DataFrame({
    "RowNumber":df_test['RowNumber'],
    "Exited":Y_pred
})
df_Clf_out.to_csv('keras_Clf.csv')

D:\anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [8]:
val_loss,val_accuracy = model.evaluate(X_test, Y_test, batch_size=1 )

1600/1600 [==============================] - 2s 1ms/step - loss: 0.3678 - accuracy: 0.8487


In [9]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                144       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 313
Trainable params: 313
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
print(Y_train)

[0 0 0 ... 0 0 1]


In [20]:
get_feature = keras.backend.function([model.layers[0].input],model.layers[1].get_output_at(0))
FC_train_feature = get_feature(X_train)
print(type(X_train))
FC_test_feature = get_feature(X_test)
fin_test=get_feature(dx_test)
print(X_train[0])
print(dx_test[0])
svc(FC_train_feature,Y_train,FC_test_feature,Y_test,fin_test)
print(FC_test_feature)

print(Y_train)

<class 'numpy.ndarray'>
[ 0.22336871 -0.13148003  1.91142144 -1.21451194  0.65037001  0.96680078
 -1.08905069 -1.009798    1.07884971 -0.16618427 -0.07850316]
[ 1.60918292 -0.50845629 -1.22234594 -1.27272998 -1.58976    -1.01613007
  0.26358415  0.95596841 -0.8981563  -0.16176299 -0.07436678]
Start training SVM...
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
cnn-svm Accuracy: 0.863125
[0 0 1 ... 1 0 0]
[[0.82280695 0.         0.         ... 0.         1.5060053  1.7509731 ]
 [0.         0.         0.         ... 0.         1.4504174  0.        ]
 [0.07747924 0.23528948 0.         ... 0.59570706 0.8112339  0.        ]
 ...
 [1.1954606  1.0939783  0.         ... 0.         0.20166558 0.        ]
 [0.92120296 0.         0.         ... 0.         0.10892439 0.        ]
 [0.         0.01028597 0.         ... 0.         2.970756   0.        ]]
[0 0 1 ... 0 0 0]
